# Nuevos listados

En esta notebook, vamos a generar los nuevos listados de palabras...

In [1]:
%matplotlib inline

import pandas as pd

analizadas = pd.read_csv("../data/listado_definitivo.csv")
analizadas.set_index(' ', inplace=True)
analizadas["candidata"] = analizadas["Palabra Candidata"]

In [2]:
palabras_ya_analizadas = analizadas[analizadas['candidata'].notna()]

print("Tenemos {} palabras analizadas".format(palabras_ya_analizadas.shape[0]))

Tenemos 5014 palabras analizadas


## Listado de Palabras

Vamos a regenerar el listado completo de palabras. Y calculemos cuánto falta trabajar si hacemos de nuevo los cálculos

In [3]:
import re

df = pd.read_csv("../output/provinces_words.csv", index_col=0)

cant_palabras = [c for c in df.columns if re.match(r'.*ocurrencias$', c)]
cant_personas = [c for c in df.columns if re.match(r'.*usuarios$', c)]

print("columnas de palabras => {}\n".format(cant_palabras))
print("columnas de usuarios => {}".format(cant_personas))

df["cant_provincias"] = (df[cant_palabras] > 0).sum(axis=1)
df["cant_palabra"] = df[cant_palabras].sum(axis=1)
df["cant_usuarios"] = df[cant_personas].sum(axis=1)

columnas de palabras => ['buenosaires_ocurrencias', 'catamarca_ocurrencias', 'chaco_ocurrencias', 'chubut_ocurrencias', 'cordoba_ocurrencias', 'corrientes_ocurrencias', 'entrerios_ocurrencias', 'formosa_ocurrencias', 'jujuy_ocurrencias', 'lapampa_ocurrencias', 'larioja_ocurrencias', 'mendoza_ocurrencias', 'misiones_ocurrencias', 'neuquen_ocurrencias', 'rionegro_ocurrencias', 'salta_ocurrencias', 'sanjuan_ocurrencias', 'sanluis_ocurrencias', 'santacruz_ocurrencias', 'santafe_ocurrencias', 'santiago_ocurrencias', 'tierradelfuego_ocurrencias', 'tucuman_ocurrencias']

columnas de usuarios => ['buenosaires_usuarios', 'catamarca_usuarios', 'chaco_usuarios', 'chubut_usuarios', 'cordoba_usuarios', 'corrientes_usuarios', 'entrerios_usuarios', 'formosa_usuarios', 'jujuy_usuarios', 'lapampa_usuarios', 'larioja_usuarios', 'mendoza_usuarios', 'misiones_usuarios', 'neuquen_usuarios', 'rionegro_usuarios', 'salta_usuarios', 'sanjuan_usuarios', 'sanluis_usuarios', 'santacruz_usuarios', 'santafe_usuario

In [4]:
df = df[(df.cant_palabra > 40) & (df.cant_usuarios >= 3) ]

Calculemos el valor de la información de cada palabra.

In [5]:
from scipy.stats import entropy
import numpy as np

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = np.log2(23) - df["entropy_palabras"]
df["delta_personas"] = np.log2(23) - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cant_palabra)
df["log_personas"] = np.log2(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]
df["ival_norm"] = (1+df["ival_palabras"]) * (1+df["ival_personas"])

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)


Cambiamos ligeramente el cálculo de la normalización y del IV, contemplando el orden de personas

¿De las primeras 1000, cuántas no están en las ya analizadas?

In [6]:
def calcular_porcentaje_candidatas(palabras):
    subconj_analizado = palabras.index.intersection(palabras_ya_analizadas.index)
    candidatas = (analizadas.loc[subconj_analizado]['candidata'] != '0').sum()
    total = subconj_analizado.shape[0]
    print("Palabras candidatas = {}".format(candidatas))
    print("Total analizadas actualmente = {}".format(total))
    
    return candidatas / total

def analizar_palabras_restantes(n, dataframes):
    total = 0
    todas_las_palabras = set()
    print("="* 80)
    print("Primeras {} palabras".format(n))
    
    for nombre, this_df in dataframes.items(): 
        print('#'*40)
        print("Métrica: {}".format(nombre))
        palabras = this_df.iloc[:n]
        
        metrica = calcular_porcentaje_candidatas(palabras)
        restantes = {p for p in palabras.index if p not in palabras_ya_analizadas.index}
        total+=len(restantes)
        todas_las_palabras = todas_las_palabras.union(restantes)
        print("Faltan etiquetar {} palabras".format(len(restantes)))
        print("Porcentaje candidatas = {}".format(metrica))
    print("Faltan etiquetar (sin repeticiones) = {}".format(len(todas_las_palabras)))

In [7]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp,
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 196
Total analizadas actualmente = 767
Faltan etiquetar 233 palabras
Porcentaje candidatas = 0.25554106910039115
########################################
Métrica: Personas
Palabras candidatas = 262
Total analizadas actualmente = 788
Faltan etiquetar 212 palabras
Porcentaje candidatas = 0.33248730964467005
########################################
Métrica: PalPer
Palabras candidatas = 248
Total analizadas actualmente = 861
Faltan etiquetar 139 palabras
Porcentaje candidatas = 0.2880371660859466
Faltan etiquetar (sin repeticiones) = 417
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 840
Total analizadas actualmente = 2047
Faltan etiquetar 2953 palabras
Porcentaje candidatas = 0.41035661944308743
########################################
Métrica: PalPer
Palabras candidatas = 828
Total analizadas actualmente = 2248
Faltan etiquetar 2

## Sin usar el log(23)

In [8]:
from contrastes.information_value import simulated_shuffled_entropy_multinomial
np.random.seed(seed=1)

fn = lambda ws: simulated_shuffled_entropy_multinomial(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)

df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cant_palabra)
df["log_personas"] = np.log2(df.cant_usuarios)

df["norm_cantidad"] = df["log_cantidad"] / df["log_cantidad"].max() 
df["norm_personas"] = df["log_personas"] / df["log_personas"].max()

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas
df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [9]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 161
Total analizadas actualmente = 735
Faltan etiquetar 265 palabras
Porcentaje candidatas = 0.21904761904761905
########################################
Métrica: Personas
Palabras candidatas = 198
Total analizadas actualmente = 693
Faltan etiquetar 307 palabras
Porcentaje candidatas = 0.2857142857142857
########################################
Métrica: PalPer
Palabras candidatas = 201
Total analizadas actualmente = 780
Faltan etiquetar 220 palabras
Porcentaje candidatas = 0.25769230769230766
Faltan etiquetar (sin repeticiones) = 517
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 795
Total analizadas actualmente = 2194
Faltan etiquetar 2806 palabras
Porcentaje candidatas = 0.3623518687329079
########################################
Métrica: PalPer
Palabras candidatas = 721
Total analizadas actualmente = 2213
Faltan etiquetar 27

## Usando cálculo con normalización "vieja"


In [10]:
from contrastes.information_value import simulated_shuffled_entropy_multinomial
np.random.seed(seed=1)

fn = lambda ws: simulated_shuffled_entropy_multinomial(int(ws), len(cant_palabras))

df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)
df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)


In [11]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cant_palabra)
df["log_personas"] = np.log2(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.norm_cantidad * df.delta_palabras
df["ival_personas"] = df.norm_personas * df.delta_personas


df["ival"] = df["ival_palabras"] * df["ival_personas"]

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [12]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 219
Total analizadas actualmente = 789
Faltan etiquetar 211 palabras
Porcentaje candidatas = 0.27756653992395436
########################################
Métrica: Personas
Palabras candidatas = 232
Total analizadas actualmente = 750
Faltan etiquetar 250 palabras
Porcentaje candidatas = 0.30933333333333335
########################################
Métrica: PalPer
Palabras candidatas = 270
Total analizadas actualmente = 907
Faltan etiquetar 93 palabras
Porcentaje candidatas = 0.29768467475192945
Faltan etiquetar (sin repeticiones) = 450
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 918
Total analizadas actualmente = 2388
Faltan etiquetar 2612 palabras
Porcentaje candidatas = 0.3844221105527638
########################################
Métrica: PalPer
Palabras candidatas = 1278
Total analizadas actualmente = 3184
Faltan etiquetar 1

## Usando el cálculo viejo CASI idéntico

No usó normalización en las dos primeras métricas

In [13]:

from contrastes.information_value import simulated_shuffled_entropy_multinomial
np.random.seed(seed=1)

fn = lambda ws: simulated_shuffled_entropy_multinomial(int(ws), len(cant_palabras))


df["sh_entropy_personas"] = df.cant_usuarios.apply(fn)
df["sh_entropy_palabras"] = df.cant_palabra.apply(fn)

In [14]:
df["entropy_palabras"] = df[cant_palabras].apply(entropy, axis=1, raw=True)
df["entropy_personas"] = df[cant_personas].apply(entropy, axis=1, raw=True)

df["delta_palabras"] = df["sh_entropy_palabras"] - df["entropy_palabras"]
df["delta_personas"] = df["sh_entropy_personas"] - df["entropy_personas"]

df["log_cantidad"] = np.log2(df.cant_palabra)
df["log_personas"] = np.log2(df.cant_usuarios)

df["norm_cantidad"] = (df["log_cantidad"] - df["log_cantidad"].min()) \
    / (df["log_cantidad"].max() - df["log_cantidad"].min())
df["norm_personas"] = (df["log_personas"] - df["log_personas"].min()) \
    / (df["log_personas"].max() - df["log_personas"].min())

df["ival_palabras"] = df.log_cantidad * df.delta_palabras
df["ival_personas"] = df.log_personas * df.delta_personas
df["ival"] = df.norm_cantidad * df.norm_personas * df.delta_palabras * df.delta_personas

df_personas = df.sort_values("ival_personas", ascending=False)
df_palabras = df.sort_values("ival_palabras", ascending=False)
df_pp = df.sort_values("ival", ascending=False)

In [15]:
dataframes = {
    "Palabras": df_palabras, 
    "Personas": df_personas, 
    "PalPer": df_pp
}
analizar_palabras_restantes(1000, dataframes)
dataframes = {
    "Personas": df_personas, 
    "PalPer": df_pp,
    
}

analizar_palabras_restantes(5000, dataframes)


Primeras 1000 palabras
########################################
Métrica: Palabras
Palabras candidatas = 160
Total analizadas actualmente = 736
Faltan etiquetar 264 palabras
Porcentaje candidatas = 0.21739130434782608
########################################
Métrica: Personas
Palabras candidatas = 198
Total analizadas actualmente = 693
Faltan etiquetar 307 palabras
Porcentaje candidatas = 0.2857142857142857
########################################
Métrica: PalPer
Palabras candidatas = 269
Total analizadas actualmente = 906
Faltan etiquetar 94 palabras
Porcentaje candidatas = 0.2969094922737307
Faltan etiquetar (sin repeticiones) = 514
Primeras 5000 palabras
########################################
Métrica: Personas
Palabras candidatas = 791
Total analizadas actualmente = 2186
Faltan etiquetar 2814 palabras
Porcentaje candidatas = 0.3618481244281793
########################################
Métrica: PalPer
Palabras candidatas = 1283
Total analizadas actualmente = 3187
Faltan etiquetar 181